In [1]:
import math
import numpy as np
import pandas as pd

In [2]:
pd.set_option("display.max_columns", None)

# Read in data

In [3]:
frames = pd.read_csv('../data/train.csv')

C:\Users\ryanc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
frames.shape

(682154, 49)

In [5]:
frames.head()

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,DisplayName,JerseyNumber,Season,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,496723,Eric Berry,29,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,212,12/29/1988,Tennessee,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,2495116,Allen Bailey,97,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,288,03/25/1989,Miami,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,2495493,Justin Houston,50,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,270,01/21/1989,Georgia,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,2506353,Derrick Johnson,56,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,245,11/22/1982,Texas,ILB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,2530794,Ron Parker,38,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,206,08/17/1987,Newberry,FS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW


# Transform Data

In [6]:
map_abbr = {'ARI': 'ARZ', 'BAL': 'BLT', 'CLE': 'CLV', 'HOU': 'HST'}
for abb in frames['PossessionTeam'].unique():
    map_abbr[abb] = abb

In [7]:
frames['PossessionTeam'] = frames['PossessionTeam'].map(map_abbr)
frames['HomeTeamAbbr'] = frames['HomeTeamAbbr'].map(map_abbr)

In [8]:
frames['IsRusher'] = (frames['NflId'] == frames['NflIdRusher']).astype(int)

In [9]:
def touchdown_distance(row):
    return (100 - row['YardLine']) if (row['PossessionTeam'] == row['FieldPosition']) else row['YardLine']

def safety_distance(row):
    return 100 - row['YardsToTouchdown']
    
frames['YardsToTouchdown'] = frames.apply(touchdown_distance, axis=1)
frames['YardsToSafety'] = frames.apply(safety_distance, axis=1)

In [11]:
frames.loc[frames.Season == 2017, 'S'] = 10*frames.loc[frames.Season == 2017,'Dis']

In [12]:
def is_offensive_player(team, poss_team, home_team):
    if team == 'away':
        if poss_team != home_team:
            return 1
        else:
            return 0
    else:
        if poss_team == home_team:
            return 1
        else:
            return 0

frames['OffensivePlayer'] = frames[['Team','PossessionTeam','HomeTeamAbbr']].apply(lambda x: is_offensive_player(x[0],x[1],x[2]), axis=1) 

In [13]:
frames['PlayLeft'] = (frames['PlayDirection'] == 'left').astype(int)

# the field is 120 yards in the x direction and 53.3 in the y direction
# the lower left corner is (0,0)
# the lower right corner is (120,0)
# the upper left corner is (0,53.3)
# the upper right corner is (120,53.3)
# if a play is going left we want to reorient to go right
# basically all of our plays should be going in the same direction
# changing up X and Y is rather simple as we just flip our coordinates
# for example, if a play is going left and the X is at 80 that just means
# we need to reorient the play right at the 40 yard line (120 - 80 = 40)
# similarly done across the field with regard to it being 53.3 yards wide
frames['X_std'] = frames['X']
frames.loc[frames.PlayLeft == 1, 'X_std'] = 120 - frames.loc[frames.PlayLeft == 1, 'X']

frames['Y_std'] = frames['Y']
frames.loc[frames.PlayLeft == 1, 'Y_std'] = 53.3 - frames.loc[frames.PlayLeft == 1, 'Y']

# direction is a bit more complicated
# it's a little easier to think about it in terms of
# a rusher going up the field vs backwards and left vs right
# we are making sure all plays are oriented from left to right
# so we have to convert the 0-360 degree orientation of plays
# going left to correspond to the newly flipped orientation going right.

# For plays going left we have to add 180 degrees to the Dir field for rushes going backwards
# and subtract 180 degrees to the Dir field for plays going forwards
# As an example, if a play is going left and the rusher runs backwards to his right at 45 degrees
# that needs to get converted from 45 degrees to 225 degrees. This corresponds to a rusher going
# backwards and to his right in the orientation going from left to right down the field.
# On the other hand, for a play going left and the rusher going forward and to his left at 225 degrees
# needs to get converted to 45 degrees to mark the new orientation going forward and left in the new rightly direction.
frames.loc[(frames.PlayLeft == 1) & (frames.Dir < 180), 'Dir_std'] = frames.Dir + 180
frames.loc[(frames.PlayLeft == 1) & (frames.Dir >= 180), 'Dir_std'] = frames.Dir - 180
# plays going right don't change direction orientation
frames.loc[frames.PlayLeft == 0, 'Dir_std'] = frames.Dir
# we need to fill in the 28 missing Dir records
frames['Dir_std'] = frames['Dir_std'].fillna(np.mean(frames['Dir_std']))

# Let's go ahead and re-orient our angles such that they are relative to the positive
# x-axis and going counter clockwise. So 0 degrees would be the running back
# rushing directly forward from left to right down the field and 180 degrees
# would be rushing directly backwards. I'm making this change so that 
# it better aligns with what you'll find with trig resources and especially
# for solving for the slope of a line based on the tan(angle) relative to x-axis.
# Q1: 90 - x; [0, 90] --> [90, 0]
# Q2: 360 - x + 90; [90, 180] --> [360, 270]
# Q3: 360 - x + 90; [180, 270] --> [270, 180]
# Q4: 360 - x + 90; [270, 360] --> [180, 90]
def reorient_angle(d):
    if d >= 0 and d <= 90:
        return 90 - d
    else:
        return 360 - d + 90
frames['Dir_std'] = frames['Dir_std'].apply(lambda x: reorient_angle(x))
frames['Dir_rads'] = frames['Dir_std']*(np.pi/180)

In [14]:
frames.to_csv('../data/cleaned_train.csv', header=True, index=False)